<a href="https://colab.research.google.com/github/brenoskuk/recvis/blob/main/SinGan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SinGan project

## MVA RecVis

### Collaborators: Breno Skuk and Claudia Torres Pérez

In [1]:
import numpy as np

In [2]:
!git clone https://github.com/tamarott/SinGAN.git

Cloning into 'SinGAN'...
remote: Enumerating objects: 790, done.
remote: Total 790 (delta 0), reused 0 (delta 0), pack-reused 790
Receiving objects: 100% (790/790), 94.89 MiB | 51.79 MiB/s, done.
Resolving deltas: 100% (304/304), done.


We need to change into the correct directory, then we install the required libraries

In [3]:
%cd SinGAN

/content/SinGAN


In [4]:
!python -m pip install -r requirements.txt

In [5]:
!python main_train.py --input_name mountains3.png

Random Seed:  5177
GeneratorConcatSkip2CleanAdd(
  (head): ConvBlock(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (LeakyRelu): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (body): Sequential(
    (block1): ConvBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (LeakyRelu): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (block2): ConvBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (LeakyRelu): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (block3): ConvBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   